In [45]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re
import time
from random import randint
import pymongo
from pymongo import MongoClient

In [37]:
# Web scrape the YELP website to pull information related to Resturant Name, Cuisine Style, Price, and Rating
restaurant_list = []

url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=West%20Hollywood%2C%20CA&start="
for page in range(0,110,10):
    response = get(url+str(page))
    html = response.text
    soup = BeautifulSoup(html,"html.parser")

    rl = re.compile("list*")
    restaurant_ul = soup.find_all('ul', class_=rl)
    for r_ul in restaurant_ul:
        border = re.compile("border*")
        restaurant_li = r_ul.find_all('li', class_=border)

        for restaurant in restaurant_li:
            container = re.compile("container*")
            aux = restaurant.find_all('div', class_=container)

            if len(aux) == 3:
                restaurant_dict = {}
                
                name = re.compile("businessNam*")
                restaurant_name = restaurant.find_all('div', class_=name)
                try:
                    restaurant_dict['name'] = restaurant_name[0].find('a').text     
                except:
                    restaurant_dict['name'] = None

                stars = re.compile("i-stars*")
                restaurant_stars = restaurant.find_all('div', class_=stars)
                try:
                    restaurant_dict['stars'] = restaurant_stars[0]["aria-label"]
                except:
                    restaurant_dict['stars'] = None
                    
                price = re.compile("priceRange*")
                restaurant_price = restaurant.find_all('span', class_=price)
                try:
                    restaurant_dict['price'] = restaurant_price[0].text
                except:
                    restaurant_dict['price'] = None
                    
                
                cuisine = re.compile("text__09f24__2NHRu css-1hx6l2b*")
                restaurant_cuisine = restaurant.find_all('p', class_=cuisine)
                try:
                    restaurant_dict['cuisine'] = restaurant_cuisine[0].text
                except:
                    restaurant_dict['cuisine'] = None
                    
                restaurant_list.append(restaurant_dict)
                    
    time.sleep(randint(1,5))
    
restaurant_list

[{'name': 'GRANVILLE',
  'stars': '4.5 star rating',
  'price': '$$',
  'cuisine': 'American (New)'},
 {'name': 'Soulmate.',
  'stars': '4 star rating',
  'price': '$$$',
  'cuisine': 'Mediterranean'},
 {'name': 'Employees Only',
  'stars': '4 star rating',
  'price': '$$$',
  'cuisine': 'Cocktail Bars'},
 {'name': 'Tu Madre - West Hollywood',
  'stars': '4 star rating',
  'price': '$$',
  'cuisine': 'Mexican'},
 {'name': 'Norah',
  'stars': '4 star rating',
  'price': '$$',
  'cuisine': 'American (New)'},
 {'name': 'The Front Yard',
  'stars': '4 star rating',
  'price': '$$',
  'cuisine': 'Tapas/Small Plates'},
 {'name': 'Republique',
  'stars': '4 star rating',
  'price': '$$',
  'cuisine': 'French'},
 {'name': 'Rosaline',
  'stars': '4 star rating',
  'price': '$$$',
  'cuisine': 'Peruvian'},
 {'name': 'Zinqué',
  'stars': '4 star rating',
  'price': '$$',
  'cuisine': 'French'},
 {'name': 'Gracias Madre - West Hollywood',
  'stars': '4 star rating',
  'price': '$$',
  'cuisine': '

In [38]:
# Create a dataframe
restaurant_df = pd.DataFrame(restaurant_list)

restaurant_df

,name,stars,price,cuisine
0,GRANVILLE,4.5 star rating,$$,American (New)
1,Soulmate.,4 star rating,$$$,Mediterranean
2,Employees Only,4 star rating,$$$,Cocktail Bars
3,Tu Madre - West Hollywood,4 star rating,$$,Mexican
4,Norah,4 star rating,$$,American (New)
...,...,...,...,...
105,Eveleigh,3.5 star rating,$$,American (New)
106,Gigi’s,4 star rating,None,French
107,Bossa Nova Brazilian Cuisine,3.5 star rating,$$,Latin American
108,n10 Los Angeles,4 star rating,$$$,Italian


In [39]:
# Convert the names in the data frame
result_df = restaurant_df[["name","stars","price","cuisine"]]

In [40]:
rating = result_df["stars"].replace("star rating","", regex=True)
result_df["star rating"] = rating

In [41]:
price_1 = result_df["price"].replace("$",1)
result_df["price"] = price_1

price_2 = result_df["price"].replace("$$", 2)
result_df["price"] = price_2

price_3 = result_df["price"].replace("$$$",3)
result_df["price"] = price_3

price_4 = result_df["price"].replace("$$$$",4)
result_df["price"] = price_4

In [42]:
result_df['rating'] = result_df['star rating'].astype(float)

In [44]:
result_clean_df = result_df[["name", "cuisine", "price", "rating"]]

result_clean_df

,name,cuisine,price,rating
0,GRANVILLE,American (New),2.0,4.5
1,Soulmate.,Mediterranean,3.0,4.0
2,Employees Only,Cocktail Bars,3.0,4.0
3,Tu Madre - West Hollywood,Mexican,2.0,4.0
4,Norah,American (New),2.0,4.0
...,...,...,...,...
105,Eveleigh,American (New),2.0,3.5
106,Gigi’s,French,NaN,4.0
107,Bossa Nova Brazilian Cuisine,Latin American,2.0,3.5
108,n10 Los Angeles,Italian,3.0,4.0


In [46]:
# Making a connection with MongoClient
client = MongoClient("mongodb://localhost:27017/")

# Create Database
db = client["YELP_Worth_It"]

# Create Collection
collection = db["CSV Data"]

In [72]:
# Convert dataframe into dictionary
results_dict = result_clean_df.to_dict('index')

results_dict

{0: {'name': 'GRANVILLE',
  'cuisine': 'American (New)',
  'price': 2.0,
  'rating': 4.5},
 1: {'name': 'Soulmate.',
  'cuisine': 'Mediterranean',
  'price': 3.0,
  'rating': 4.0},
 2: {'name': 'Employees Only',
  'cuisine': 'Cocktail Bars',
  'price': 3.0,
  'rating': 4.0},
 3: {'name': 'Tu Madre - West Hollywood',
  'cuisine': 'Mexican',
  'price': 2.0,
  'rating': 4.0},
 4: {'name': 'Norah',
  'cuisine': 'American (New)',
  'price': 2.0,
  'rating': 4.0},
 5: {'name': 'The Front Yard',
  'cuisine': 'Tapas/Small Plates',
  'price': 2.0,
  'rating': 4.0},
 6: {'name': 'Republique', 'cuisine': 'French', 'price': 2.0, 'rating': 4.0},
 7: {'name': 'Rosaline', 'cuisine': 'Peruvian', 'price': 3.0, 'rating': 4.0},
 8: {'name': 'Zinqué', 'cuisine': 'French', 'price': 2.0, 'rating': 4.0},
 9: {'name': 'Gracias Madre - West Hollywood',
  'cuisine': 'Mexican',
  'price': 2.0,
  'rating': 4.0},
 10: {'name': 'Conservatory',
  'cuisine': 'American (New)',
  'price': 3.0,
  'rating': 4.0},
 11: {'

In [73]:
# Insert data into MongoDB
for result in results_dict.values():
    collection.insert_one(result)